In [2]:
import datetime
from datetime import datetime
from datetime import timedelta
from typing import List, Union, Dict
import pickle
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

# Import
df = pd.read_csv("~/CNNworkspace/testdataDEC/nomil_cleanstitch.csv")

# New Time - Concatenate Date & Time fields into 1 proper datetime field
# # making a DateTime column of the datetime type for easier manipulation
# df["DateTime"] = df.apply(
#     lambda x: datetime.strptime(f"{x['date']} {x['time']}", "%d/%m/%Y %H:%M:%S"),
#     axis=1,
# )

# # Remove bad Date & Time columns
#df = df.drop(["date", "time"], axis=1)

# # Get a group count to give us an anchor for iterative millisecond additions

#df["annotation_group_step"] = df.groupby("date_time").cumcount()
df.head(10)



,behavior,acc_x,acc_y,acc_z,tag_id,date,time,camera_time,temp_c,battery_voltage,metadata,input_index
0,s,0.047,0.141,0.953,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,0
1,s,0.047,0.125,0.938,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,1
2,s,0.047,0.141,0.953,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,2
3,s,0.047,0.141,0.938,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,3
4,s,0.063,0.141,0.953,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,24.0,NaN,NaN,4
5,s,0.047,0.125,0.938,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,5
6,s,0.063,0.141,0.953,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,6
7,s,0.063,0.125,0.938,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,7
8,s,0.063,0.141,0.938,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,8
9,s,0.047,0.141,0.938,JLTA_AXY#7_S1,19/05/2021,10:57:08,1:58:42,NaN,NaN,NaN,9


# Get behavior counts

In [22]:
def min_count(df):
    """Input: df -- dataframe of cleaned data
    output: min_val -- single integer inside a list of the smallest number of instances"""
    #TODO: must verify that the min values are the same behavior and continuous (index vals)
    bcounts = df['behavior'].value_counts()
    bdict = bcounts.to_dict() # outputs a dict with key = behavior, value = count

    min_val = [values for keys,values in bdict.items() if values == min(bdict.values())] #outputs all behaviors that have
                #the fewest rows of data. now if the list is longer than a single value, we will test both for consecutive
        #window length = the value_counts() of the min_val behavior
    windowsize = min_val[0]
    return windowsize

odata = df.behavior.unique()
data = odata.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)

onehot = encoder.fit_transform(data)
print(len(set(np.ediff1d(df.acc_x))))
print(data)
print(onehot)



97
[['s']
 ['l']
 ['t']
 ['c']
 ['a']]


NameError: name 'windowsize' is not defined

In [4]:
def one_hot_coder(df):
    total_behaviors = list(df.behavior.unique())
    num_classes = len(total_behaviors)
    mapping = {}
    for x in range(num_classes):
        mapping[total_behaviors[x]] = x
    for x in range(num_classes):
        total_behaviors[x] = mapping[total_behaviors[x]]
    #one_hot_encode = to_categorical(mapping[])
    return mapping

def one_hot_labeler(mapper, behavior):
    one_hot_label = to_categorical(mapper[behavior])
    return one_hot_label

themap = one_hot_coder(df)
print(themap)
print(one_hot_labeler(themap,'a'))

# ndf = df.head(100)
# pdf = pd.to_datetime(ndf['date_time'])
# for stamp in lambda x: datetime.timestamp(x))
# print(datetime.timestamp(pdf[1]))

#ndf['date_time'] = pd.to_datetime(df['date_time']).astype(np.int64)
#ndf['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
#ndf.int


{'s': 0, 'l': 1, 't': 2, 'c': 3, 'a': 4}
[0. 0. 0. 0. 1.]


In [25]:
def pull_windows(df, slide: int = 1, window_length: int = 15):
    """ Pull matrix window """
    
    # production 
    if window_length > df.shape[0]:
        raise ValueError('Window larger than data given')
    
    windows = []
    number_of_rows_minus_window = df.shape[0] - window_length + 1
    
    for i in range(0, number_of_rows_minus_window, slide):
        window = df[i: i + window_length]
        #V
       # dtwindow = pd.to_datetime(window['date_time'])
        #^
        #indarray = window['input_index'].values
        #print(window)
            
        # check if all behaviors are the same; if not ignore window
        if len(set(window.behavior)) != 1:
            continue
        
        # check if times are uniform -- equal length; if not ignore window
        
        if len(set(np.ediff1d(window.input_index))) != 1:
             continue
        # len(set(np.ediff1d(window.DateTime))) != 1
        
        windows.append(window)
        
    return windows

    
def contruct_train_test(windows, df):
    positions = ['acc_x', 'acc_y', 'acc_z']
    Xtrain, Ytrain = [], []  # have to be the same length
    # TODO - give each classifier its own index and the length should be the numpy of classifiers
    
    onehot = {
            's': [0, 0, 0, 1, 0],
            'l': [0, 0, 1, 0, 0],
            't': [0, 0, 0, 0, 1], 
            'c': [0, 1, 0, 0, 0], 
            'a': [1, 0, 0, 0, 0]
            }
#     {
#         'n': [0, 0, 0, 1],  # Example: there there is only 4 classifiers in this example and we assigned 'n' to index 3
#     }
    for window in windows:
        Xtrain.append(window[positions].to_numpy())
        Ytrain.append(onehot[window.behavior.values[0]])
        
    return np.stack(Xtrain), Ytrain
    
windowsize = min_count(df)
windows: List[pd.DataFrame] = pull_windows(df, slide=1, window_length=windowsize)

Xtrain, Ytrain = contruct_train_test(windows, df)
print(Xtrain[0], Ytrain[0])
print(Xtrain[1], Ytrain[1])

[[0.047 0.141 0.953]
 [0.047 0.125 0.938]
 [0.047 0.141 0.953]
 [0.047 0.141 0.938]
 [0.063 0.141 0.953]
 [0.047 0.125 0.938]
 [0.063 0.141 0.953]
 [0.063 0.125 0.938]] [0, 0, 0, 1, 0]
[[0.047 0.125 0.938]
 [0.047 0.141 0.953]
 [0.047 0.141 0.938]
 [0.063 0.141 0.953]
 [0.047 0.125 0.938]
 [0.063 0.141 0.953]
 [0.063 0.125 0.938]
 [0.063 0.141 0.938]] [0, 0, 0, 1, 0]


# Save preprocessing 

In [104]:
with open('input.pkl', 'wb') as outfile:
    pickle.dump((Xtrain, Ytrain), outfile, pickle.HIGHEST_PROTOCOL)

# Read preprocessing

In [105]:
with open('input.pkl', 'rb') as infile:
    Xtrain, Ytrain = pickle.load(infile)

Xtrain[0], Ytrain[0]

(array([[-0.016, -0.219,  0.985],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.203,  0.969],
        [-0.016, -0.203,  0.969],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.219,  0.969],
        [-0.016, -0.203,  0.985],
        [-0.016, -0.219,  0.985]]),
 [0, 0, 0, 1])

# create model 

In [ ]:
def create_model(Xtrain, *args, *kwargs):
    pass

model = create_model(Xtrain)

# train model

In [ ]:
model = model.fit(Xtrain, Ytrain)
model.tensorflow_fuckin_save

# test model 

In [87]:
def fix_ytest(Ytest) -> np.array:
    pass

windows = pull_windows(df, slide=1, window_length=10)
Xtest, Ytest = contruct_train_test(windows)

Ytest_predicted = model.predict(Xtest)  # may not be correct - tf.keras.model should have a predict function

# Yest_predicted == Ytest?
Ytest_predicted_fixed = fix(Ytest_predicted)  # i.e. a.index(max([0, .1, .22, .7]))

# Plot results

In [94]:
# yay!